In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import torchinfo

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [2]:
train_def_path = "/kaggle/input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train/def_front"
train_ok_path = "/kaggle/input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train/ok_front"
val_def_path = "/kaggle/input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/def_front"
val_ok_path = "/kaggle/input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/ok_front"

In [3]:
class Data(Dataset):
    def __init__(self, def_path, ok_path):
        self.def_path = def_path
        self.ok_path = ok_path
        self.def_imgs = np.array([os.path.join(self.def_path, i) for i in os.listdir(self.def_path)])
        self.ok_imgs = np.array([os.path.join(self.ok_path, i) for i in os.listdir(self.ok_path)])
        self.all_imgs = np.concatenate((self.def_imgs, self.ok_imgs))
        np.random.shuffle(self.all_imgs)
    def __len__(self):
        return len(self.all_imgs)
        
    def __getitem__(self, idx):
        img_path = self.all_imgs[idx]
        img = torch.tensor(cv.cvtColor(cv.imread(img_path), cv.COLOR_BGR2RGB)).type(torch.FloatTensor) #HWC
        img = torch.permute(img, (2,0,1)) #CHW
        if img_path in self.def_imgs: y = [0,1] #[0th index repr ok]
        elif img_path in self.ok_imgs: y = [1,0]
        return img, torch.tensor(y).type(torch.FloatTensor)

train_data = Data(train_def_path, train_ok_path)
val_data = Data(val_def_path, val_ok_path)`

In [4]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3),
            nn.Dropout2d(p=0.2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(num_features = 1024),
            nn.Conv2d(in_channels=1024, out_channels=128, kernel_size=1),
            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.LazyLinear(out_features=256),
            nn.Sigmoid(),
            nn.Linear(in_features=256, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features = 128, out_features=64),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=2),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return self.net(x)

model = Classifier()
print(torchinfo.summary(model, (1, 3, 300, 300)))

c:\Python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Layer (type:depth-idx)                   Output Shape              Param #
Classifier                               [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─Conv2d: 2-1                       [1, 32, 298, 298]         896
│    └─ReLU: 2-2                         [1, 32, 298, 298]         --
│    └─Conv2d: 2-3                       [1, 64, 296, 296]         18,496
│    └─ReLU: 2-4                         [1, 64, 296, 296]         --
│    └─MaxPool2d: 2-5                    [1, 64, 148, 148]         --
│    └─Conv2d: 2-6                       [1, 128, 146, 146]        73,856
│    └─ReLU: 2-7                         [1, 128, 146, 146]        --
│    └─Conv2d: 2-8                       [1, 256, 144, 144]        295,168
│    └─ReLU: 2-9                         [1, 256, 144, 144]        --
│    └─MaxPool2d: 2-10                   [1, 256, 72, 72]          --
│    └─Conv2d: 2-11                      [1, 512, 70, 70]          1,18

In [5]:
LR = 5e-2
LOSS_FN = nn.CrossEntropyLoss()
OPTIM = torch.optim.SGD(model.parameters(), lr=LR)
EPOCHS = 50
BATCH_SIZE = 32

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

model = model.to(device)


epochwise_train_losses = []
epochwise_val_losses = []
for epoch in range(EPOCHS):
    batchwise_train_losses = []
    batchwise_val_losses = []
    print(f'{"-"*20}Epoch: {epoch+1}/{EPOCHS}{"-"*20}')
    model.train()    
    for batch, (x,y) in enumerate(train_loader):
        x,y = x.to(device), y.to(device)
        x = x/255
        pred = model(x)
        loss = LOSS_FN(pred, y)
        loss.backward()
        OPTIM.step()
        OPTIM.zero_grad()
        batchwise_train_losses.append(loss.item())
    epoch_train_loss = np.mean(batchwise_train_losses)
    epochwise_train_losses.append(epoch_train_loss)
    print(f'Training Loss: {epoch_train_loss:.5f}')
    
    model.eval()
    for batch, (x,y) in enumerate(val_loader):
        x,y = x.to(device), y.to(device)
        x = x/255
        pred = model(x)
        loss = LOSS_FN(pred, y)
        batchwise_val_losses.append(loss.item())
    epoch_val_loss = np.mean(batchwise_val_losses)
    epochwise_val_losses.append(epoch_val_loss)
    print(f'Validation Loss: {epoch_val_loss:.5f}')

In [6]:
plt.plot(epochwise_train_losses, label='train-loss')
plt.plot(epochwise_val_losses, label='validation-loss')
plt.legend()
plt.show()

In [7]:
torch.save(model, 'model.pt')